In [294]:
import pandas as pd
import numpy as np
import ast
import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from PIL import Image
import tensorflow as tf
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Dense, Input

# Data Pre-Processing

### Load and Merge Datasets

In [295]:
# Load Datasets

# Training datasets
cgm_train = pd.read_csv('cgm_train.csv')
image_train = pd.read_csv('img_train.csv')
demo_viome_train = pd.read_csv('demo_viome_train.csv')
label_train = pd.read_csv('label_train.csv')

# Test datasets
cgm_test = pd.read_csv('cgm_test.csv')
image_test = pd.read_csv('img_test.csv')
demo_viome_test = pd.read_csv('demo_viome_test.csv')
label_test = pd.read_csv('label_test_breakfast_only.csv')

In [296]:
# Merge CGM and Image datasets
temp_data = pd.merge(image_train, cgm_train, on=['Subject ID', 'Day'])
output_labels = label_train[["Subject ID", "Day", "Lunch Calories"]]
data_train = pd.merge(temp_data, output_labels, on=['Subject ID','Day'])

In [297]:
data_train

,Subject ID,Day,Image Before Breakfast,Image Before Lunch,Breakfast Time,Lunch Time,CGM Data,Lunch Calories
0,1,2,"[[[140, 122, 108], [135, 118, 104], [118, 104,...","[[[41, 152, 201], [77, 164, 205], [88, 157, 13...",2021-09-19 08:41:00,2021-09-19 12:24:00,"[('2021-09-19 08:20:00', 98.26666666666667), (...",830
1,1,3,"[[[67, 58, 47], [59, 52, 41], [51, 45, 35], [4...","[[[40, 59, 77], [35, 56, 72], [20, 36, 47], [9...",2021-09-20 09:50:00,2021-09-20 15:20:00,"[('2021-09-20 09:10:00', 97.18333333333334), (...",435
2,1,4,"[[[199, 195, 193], [198, 193, 192], [196, 192,...","[[[53, 44, 38], [51, 43, 36], [54, 47, 39], [4...",2021-09-21 09:34:00,2021-09-21 13:09:00,"[('2021-09-21 09:20:00', 107.36666666666666), ...",555
3,1,5,"[[[149, 121, 80], [157, 128, 86], [159, 130, 8...","[[[30, 28, 28], [20, 18, 17], [31, 27, 23], [2...",2021-09-22 09:46:00,2021-09-22 13:50:00,"[('2021-09-22 09:25:00', 107.28333333333333), ...",355
4,1,6,"[[[175, 184, 198], [192, 206, 219], [160, 165,...","[[[74, 85, 100], [59, 69, 81], [73, 84, 96], [...",2021-09-23 09:07:00,2021-09-23 13:17:00,"[('2021-09-23 08:55:00', 103.0), ('2021-09-23 ...",1180
...,...,...,...,...,...,...,...,...
319,7,6,"[[[68, 34, 35], [82, 60, 51], [63, 55, 38], [3...","[[[90, 77, 75], [92, 78, 75], [94, 83, 81], [9...",2021-12-18 08:52:00,2021-12-18 12:28:00,"[('2021-12-18 08:50:00', 101.36), ('2021-12-18...",1180
320,7,7,"[[[26, 26, 22], [17, 17, 13], [18, 19, 14], [9...","[[[17, 9, 8], [10, 7, 7], [3, 3, 4], [3, 3, 3]...",2021-12-19 08:43:00,2021-12-19 13:13:00,"[('2021-12-19 08:40:00', 100.68), ('2021-12-19...",830
321,7,8,"[[[43, 37, 33], [42, 36, 31], [42, 37, 33], [4...","[[[122, 108, 107], [124, 110, 108], [124, 111,...",2021-12-20 09:06:00,2021-12-20 12:46:00,"[('2021-12-20 09:00:00', 104.04), ('2021-12-20...",435
322,7,9,"[[[41, 38, 33], [41, 38, 33], [41, 38, 33], [4...","[[[59, 46, 32], [63, 51, 41], [57, 42, 28], [6...",2021-12-21 08:34:00,2021-12-21 12:38:00,"[('2021-12-21 08:25:00', 96.4), ('2021-12-21 0...",555


### Pre-Process Food Pictures (Image Dataset) - COMPLETE

Replaces missing images with blank image.  
Resizes and normalizes image.

In [298]:
# Placeholder for missing images (a blank black image)
def create_placeholder_image(size=(64, 64, 3)):
    return np.zeros(size, dtype=np.float32)  # Normalized [0, 1] range

# Function to preprocess image data
def handle_image(img_data, size=(64, 64)):
    try:
        img_array = np.array(img_data, dtype=np.uint8)  # Ensure valid data type

        # Check for empty image
        if img_array.size == 0 or img_array.ndim != 3 or img_array.shape[2] != 3:
            raise ValueError(f"Invalid or empty image dimensions: {img_array.shape}")

        img_resized = np.array(Image.fromarray(img_array).resize(size))  # Resize
        img_normalized = img_resized / 255.0  # Normalize pixel values to [0, 1]
        return img_normalized
    except Exception as e:
        print(f"Error preprocessing image: {e}")
        return create_placeholder_image(size)

# Preprocess the dataset
def preprocess_img(data):
    # Define placeholder image
    placeholder_image = create_placeholder_image()

    # Iterate over rows to preprocess images
    breakfast_images = []
    lunch_images = []

    for index, row in data.iterrows():
        # Handle missing breakfast images
        if pd.isnull(row['Image Before Breakfast']) or row['Image Before Breakfast'] == '[]':  # Check for empty list or NaN
            breakfast_images.append(placeholder_image)
        else:
            try:
                img_data = eval(row['Image Before Breakfast'])  # Convert string to list
                breakfast_images.append(handle_image(img_data))
            except Exception as e:
                print(f"Error at index {index}, breakfast: {e}")
                breakfast_images.append(placeholder_image)

        # Handle missing lunch images
        if pd.isnull(row['Image Before Lunch']) or row['Image Before Lunch'] == '[]':  # Check for empty list or NaN
            lunch_images.append(placeholder_image)
        else:
            try:
                img_data = eval(row['Image Before Lunch'])  # Convert string to list
                lunch_images.append(handle_image(img_data))
            except Exception as e:
                print(f"Error at index {index}, lunch: {e}")
                lunch_images.append(placeholder_image)

    # Add preprocessed images back to the dataset
    data['Image Before Breakfast'] = breakfast_images
    data['Image Before Lunch'] = lunch_images

    return data

In [299]:
data_train = preprocess_img(data_train)

data_train

,Subject ID,Day,Image Before Breakfast,Image Before Lunch,Breakfast Time,Lunch Time,CGM Data,Lunch Calories
0,1,2,"[[[0.5490196078431373, 0.47843137254901963, 0....","[[[0.1607843137254902, 0.596078431372549, 0.78...",2021-09-19 08:41:00,2021-09-19 12:24:00,"[('2021-09-19 08:20:00', 98.26666666666667), (...",830
1,1,3,"[[[0.2627450980392157, 0.22745098039215686, 0....","[[[0.1568627450980392, 0.23137254901960785, 0....",2021-09-20 09:50:00,2021-09-20 15:20:00,"[('2021-09-20 09:10:00', 97.18333333333334), (...",435
2,1,4,"[[[0.7803921568627451, 0.7647058823529411, 0.7...","[[[0.20784313725490197, 0.17254901960784313, 0...",2021-09-21 09:34:00,2021-09-21 13:09:00,"[('2021-09-21 09:20:00', 107.36666666666666), ...",555
3,1,5,"[[[0.5843137254901961, 0.4745098039215686, 0.3...","[[[0.11764705882352941, 0.10980392156862745, 0...",2021-09-22 09:46:00,2021-09-22 13:50:00,"[('2021-09-22 09:25:00', 107.28333333333333), ...",355
4,1,6,"[[[0.6862745098039216, 0.7215686274509804, 0.7...","[[[0.2901960784313726, 0.3333333333333333, 0.3...",2021-09-23 09:07:00,2021-09-23 13:17:00,"[('2021-09-23 08:55:00', 103.0), ('2021-09-23 ...",1180
...,...,...,...,...,...,...,...,...
319,7,6,"[[[0.26666666666666666, 0.13333333333333333, 0...","[[[0.35294117647058826, 0.30196078431372547, 0...",2021-12-18 08:52:00,2021-12-18 12:28:00,"[('2021-12-18 08:50:00', 101.36), ('2021-12-18...",1180
320,7,7,"[[[0.10196078431372549, 0.10196078431372549, 0...","[[[0.06666666666666667, 0.03529411764705882, 0...",2021-12-19 08:43:00,2021-12-19 13:13:00,"[('2021-12-19 08:40:00', 100.68), ('2021-12-19...",830
321,7,8,"[[[0.16862745098039217, 0.1450980392156863, 0....","[[[0.47843137254901963, 0.4235294117647059, 0....",2021-12-20 09:06:00,2021-12-20 12:46:00,"[('2021-12-20 09:00:00', 104.04), ('2021-12-20...",435
322,7,9,"[[[0.1607843137254902, 0.14901960784313725, 0....","[[[0.23137254901960785, 0.1803921568627451, 0....",2021-12-21 08:34:00,2021-12-21 12:38:00,"[('2021-12-21 08:25:00', 96.4), ('2021-12-21 0...",555


### Pre-Process CGM Data (Time-Series Glucose Levels) - COMPLETE

Replace missing values of breakfast and lunch times with mean meal times of that subject.  
Used measures of variability for time-series-data CGM Data.  
Time between meals added as input feature.

In [300]:
# Updating missing values with mean
def preprocess_cgm(data_train):

    # Function to check if CGM Data is an empty array
    def is_cgm_data_empty(row):
        try:
            cgm_list = ast.literal_eval(row['CGM Data'])
            return len(cgm_list) == 0
        except:
            return True

    # Function to filter out rows where CGM Data is empty
    data_train = data_train[~data_train.apply(is_cgm_data_empty, axis=1)]

    # Get the HH:MM:SS breakfast and lunch times and calculate the mean for each subject ID
    data_train.loc[:, 'Breakfast Time'] = pd.to_datetime(data_train['Breakfast Time'], errors='coerce')
    data_train.loc[:, 'Lunch Time'] = pd.to_datetime(data_train['Lunch Time'], errors='coerce')

    def mean_time(times):
        total_seconds = sum([t.hour * 3600 + t.minute * 60 + t.second for t in times if pd.notna(t)])
        mean_seconds = total_seconds // len([t for t in times if pd.notna(t)])
        return pd.to_datetime(mean_seconds, unit='s').time()

    mean_times = data_train.groupby('Subject ID')[['Breakfast Time', 'Lunch Time']].apply(
        lambda group: pd.Series({
            'Breakfast Time': mean_time(group['Breakfast Time']) if not group['Breakfast Time'].isna().all()
                         else datetime.time(8, 0),
            'Lunch Time': mean_time(group['Lunch Time'])
        })
    )

    mean_times = mean_times.reset_index()

    # Find the reference date for any row within the same subject:
    def get_reference_date(subject_id):
        day_2_breakfast_index = data_train[(data_train['Subject ID'] == subject_id) & (data_train['Day'] == 2)]['Breakfast Time'].first_valid_index()
        if day_2_breakfast_index is None or pd.isna(data_train.loc[day_2_breakfast_index, 'Breakfast Time']):
            # If Breakfast Time is not available for Day 2, check Lunch Time
            day_2_lunch_index = data_train[(data_train['Subject ID'] == subject_id) & (data_train['Day'] == 2)]['Lunch Time'].first_valid_index()
            if day_2_lunch_index is not None:
                reference_date = data_train.loc[day_2_lunch_index, 'Lunch Time']
                reference_day = 2
            else:
                # If neither Breakfast nor Lunch time is available for Day 2, check Day 3
                day_3_breakfast_index = data_train[(data_train['Subject ID'] == subject_id) & (data_train['Day'] == 3)]['Breakfast Time'].first_valid_index()
                if day_3_breakfast_index is not None:
                    reference_date = data_train.loc[day_3_breakfast_index, 'Breakfast Time']
                    reference_day = 3
                else:
                    reference_date = None
                    reference_day = None
        else:
            reference_date = data_train.loc[day_2_breakfast_index, 'Breakfast Time']
            reference_day = 2
        
        return pd.Series([reference_date.date(), reference_day])

    mean_times[['Reference Date', 'Reference Day']] = mean_times['Subject ID'].apply(get_reference_date)

    # Update missing values
    def update_missing_breakfast_time(row, mean_times):
        subject_id = row['Subject ID']
        day = row['Day']
        
        mean_breakfast_time = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Breakfast Time'].iloc[0]
        mean_reference_date = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Reference Date'].iloc[0]
        reference_day = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Reference Day'].iloc[0]
        current_date = mean_reference_date + pd.Timedelta(days=(day - reference_day))
        updated_breakfast_time = pd.to_datetime(current_date.strftime('%Y-%m-%d') + ' ' + mean_breakfast_time.strftime('%H:%M:%S'))
        row['Breakfast Time'] = updated_breakfast_time
        return row

    data_train = data_train.apply(
        lambda row: update_missing_breakfast_time(row, mean_times) if pd.isna(row['Breakfast Time']) else row, axis=1
    )

    def update_missing_lunch_time(row, mean_times):
        subject_id = row['Subject ID']
        day = row['Day']
        mean_lunch_time = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Lunch Time'].iloc[0]
        mean_reference_date = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Reference Date'].iloc[0]
        reference_day = mean_times.loc[mean_times['Subject ID'] == subject_id, 'Reference Day'].iloc[0]
        current_date = mean_reference_date + pd.Timedelta(days=(day - reference_day))
        updated_lunch_time = pd.to_datetime(current_date.strftime('%Y-%m-%d') + ' ' + mean_lunch_time.strftime('%H:%M:%S'))
        row['Lunch Time'] = updated_lunch_time
        return row

    data_train = data_train.apply(
        lambda row: update_missing_lunch_time(row, mean_times) if pd.isna(row['Lunch Time']) else row, axis=1
    )

    data_train['Time Between Meals'] = (data_train['Lunch Time'] - data_train['Breakfast Time']).dt.total_seconds()

    return data_train

In [301]:
data_train = preprocess_cgm(data_train)

data_train

,Subject ID,Day,Image Before Breakfast,Image Before Lunch,Breakfast Time,Lunch Time,CGM Data,Lunch Calories,Time Between Meals
0,1,2,"[[[0.5490196078431373, 0.47843137254901963, 0....","[[[0.1607843137254902, 0.596078431372549, 0.78...",2021-09-19 08:41:00,2021-09-19 12:24:00,"[('2021-09-19 08:20:00', 98.26666666666667), (...",830,13380.0
1,1,3,"[[[0.2627450980392157, 0.22745098039215686, 0....","[[[0.1568627450980392, 0.23137254901960785, 0....",2021-09-20 09:50:00,2021-09-20 15:20:00,"[('2021-09-20 09:10:00', 97.18333333333334), (...",435,19800.0
2,1,4,"[[[0.7803921568627451, 0.7647058823529411, 0.7...","[[[0.20784313725490197, 0.17254901960784313, 0...",2021-09-21 09:34:00,2021-09-21 13:09:00,"[('2021-09-21 09:20:00', 107.36666666666666), ...",555,12900.0
3,1,5,"[[[0.5843137254901961, 0.4745098039215686, 0.3...","[[[0.11764705882352941, 0.10980392156862745, 0...",2021-09-22 09:46:00,2021-09-22 13:50:00,"[('2021-09-22 09:25:00', 107.28333333333333), ...",355,14640.0
4,1,6,"[[[0.6862745098039216, 0.7215686274509804, 0.7...","[[[0.2901960784313726, 0.3333333333333333, 0.3...",2021-09-23 09:07:00,2021-09-23 13:17:00,"[('2021-09-23 08:55:00', 103.0), ('2021-09-23 ...",1180,15000.0
...,...,...,...,...,...,...,...,...,...
319,7,6,"[[[0.26666666666666666, 0.13333333333333333, 0...","[[[0.35294117647058826, 0.30196078431372547, 0...",2021-12-18 08:52:00,2021-12-18 12:28:00,"[('2021-12-18 08:50:00', 101.36), ('2021-12-18...",1180,12960.0
320,7,7,"[[[0.10196078431372549, 0.10196078431372549, 0...","[[[0.06666666666666667, 0.03529411764705882, 0...",2021-12-19 08:43:00,2021-12-19 13:13:00,"[('2021-12-19 08:40:00', 100.68), ('2021-12-19...",830,16200.0
321,7,8,"[[[0.16862745098039217, 0.1450980392156863, 0....","[[[0.47843137254901963, 0.4235294117647059, 0....",2021-12-20 09:06:00,2021-12-20 12:46:00,"[('2021-12-20 09:00:00', 104.04), ('2021-12-20...",435,13200.0
322,7,9,"[[[0.1607843137254902, 0.14901960784313725, 0....","[[[0.23137254901960785, 0.1803921568627451, 0....",2021-12-21 08:34:00,2021-12-21 12:38:00,"[('2021-12-21 08:25:00', 96.4), ('2021-12-21 0...",555,14640.0


In [302]:
# # Function to check if CGM Data is an empty array
# def is_cgm_data_empty(row):
#     try:
#         cgm_list = ast.literal_eval(row['CGM Data'])
#         return len(cgm_list) == 0
#     except:
#         return True

# # Function to filter out rows where CGM Data is empty
# cgm_train = cgm_train[~cgm_train.apply(is_cgm_data_empty, axis=1)]

# # Handle missing breakfast and lunch times
# cgm_train['Breakfast Time'] = pd.to_datetime(cgm_train['Breakfast Time'], errors='coerce')
# cgm_train['Lunch Time'] = pd.to_datetime(cgm_train['Lunch Time'], errors='coerce')

# # Extract CGM data as list of tuples, convert to list of time series values
# cgm_train['CGM Data'] = cgm_train['CGM Data'].apply(lambda x: eval(x) if isinstance(x, str) else [])

# # Extract features from CGM data (flatten the time and glucose values)
# def extract_cgm_features(cgm_data):
#     times = [entry[0] for entry in cgm_data]
#     glucose_levels = [entry[1] for entry in cgm_data]
#     return times, glucose_levels

# cgm_train['CGM Times'], cgm_train['CGM Levels'] = zip(*cgm_train['CGM Data'].apply(extract_cgm_features))

# # Normalize glucose levels
# scaler = StandardScaler()
# cgm_train['CGM Levels'] = cgm_train['CGM Levels'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)).flatten())

# # We need to pad the sequences to a fixed length for GRU input
# max_sequence_length = 300  # Define a maximum length for the sequences
# cgm_train['Padded CGM Levels'] = pad_sequences(cgm_train['CGM Levels'], maxlen=max_sequence_length, padding='post', value=0, dtype='float32').tolist()

# # Mask labels: We will use NaN or a predefined mask value for missing times -- 1
# cgm_train['Breakfast Time Masked'] = cgm_train['Breakfast Time'].isna().astype(int)
# cgm_train['Lunch Time Masked'] = cgm_train['Lunch Time'].isna().astype(int)

# # Prepare the target variable: encode the time values for breakfast and lunch
# def encode_times(time_column):
#     return (time_column - pd.Timestamp('2021-09-18')) // pd.Timedelta('1s')

# # Filter rows where both Breakfast and Lunch times are missing (i.e., both masks are 0)
# filtered_cgm_train = cgm_train[(cgm_train['Breakfast Time Masked'] == 0) & (cgm_train['Lunch Time Masked'] == 0)].copy()

# # Encode breakfast and lunch times only for rows where both are missing
# filtered_cgm_train['Breakfast Time Encoded'] = encode_times(filtered_cgm_train['Breakfast Time'])
# filtered_cgm_train['Lunch Time Encoded'] = encode_times(filtered_cgm_train['Lunch Time'])

# time_scaler = MinMaxScaler()

# # Reshape and scale both 'Breakfast Time Encoded' and 'Lunch Time Encoded'
# filtered_cgm_train[['Breakfast Time Encoded', 'Lunch Time Encoded']] = time_scaler.fit_transform(
#     filtered_cgm_train[['Breakfast Time Encoded', 'Lunch Time Encoded']]
# )

# X_train = np.array(filtered_cgm_train['Padded CGM Levels'].tolist())
# y_train = filtered_cgm_train[['Breakfast Time Encoded', 'Lunch Time Encoded']].values

# # Reshape X_train to (samples, time steps, features)
# X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # (samples, time steps, features)

# # Define the GRU model
# def create_gru_model(input_shape):
#     model = Sequential([
#         Input(shape=input_shape),
#         GRU(32, activation='relu'),
#         Dense(2)  # Output two values: breakfast and lunch times
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.001, clipvalue=1.0), loss='mse')
#     return model

# # Create and compile the model
# model = create_gru_model((X_train_reshaped.shape[1], 1))

# # Train the model
# model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


In [303]:
# def predict_meal_times(model, X):
#     X = np.array(X.tolist()) if isinstance(X, pd.Series) else np.array(X)
#     X_reshaped = X.reshape((X.shape[0], X.shape[1], 1))
#     predictions = model.predict(X_reshaped)
#     reference_date = pd.Timestamp('2021-09-18')

#     # Decode predictions back to original scale using inverse transformation
#     decoded_times = time_scaler.inverse_transform(predictions)

#     # Add the decoded seconds back to the reference date
#     decoded_breakfast_timestamps = reference_date + pd.to_timedelta(decoded_times[:, 0], unit='s')
#     decoded_lunch_timestamps = reference_date + pd.to_timedelta(decoded_times[:, 1], unit='s')

#     decoded_predictions = pd.DataFrame({
#         'Predicted Breakfast Time': decoded_breakfast_timestamps,
#         'Predicted Lunch Time': decoded_lunch_timestamps
#     })

#     return decoded_predictions

# # Extract rows with missing breakfast or lunch times
# missing_data = cgm_train[cgm_train['Breakfast Time'].isna() | cgm_train['Lunch Time'].isna()]

# # Ensure that `Padded CGM Levels` is included in `missing_data`
# predict_missing = missing_data['Padded CGM Levels']
# missing_data_copy = missing_data.copy()

# # Make predictions for missing breakfast and lunch times
# predicted_times = predict_meal_times(model, predict_missing)

# # Reset indices for both DataFrames to align by row order
# missing_data_copy = missing_data_copy.reset_index(drop=True)
# predicted_times = predicted_times.reset_index(drop=True)

# # Add the 'Predicted Breakfast Time' column
# missing_data_copy['Predicted Breakfast Time'] = predicted_times['Predicted Breakfast Time']
# missing_data_copy['Predicted Lunch Time'] = predicted_times['Predicted Lunch Time']

### Pre-Process Viome Data (Demographic Data) - COMPLETE

Split Viome data into individual features.  
Encoded categorical value of race, gender and diabetes status.

In [304]:
def preprocess_viome(data):
    # Split the `Viome` column into individual features
    viome_split = data['Viome'].str.split(',', expand=True).astype(float)
    viome_split.columns = [f"Viome_{i}" for i in range(viome_split.shape[1])]
    data = pd.concat([data.drop(columns=['Viome']), viome_split], axis=1)

    # Impute missing values for numeric columns
    numeric_cols = data.select_dtypes(include=[np.number]).columns.drop('Subject ID')
    imputer = SimpleImputer(strategy='mean')
    data[numeric_cols] = imputer.fit_transform(data[numeric_cols])

    # Encode categorical columns
    categorical_cols = ['Race', 'Diabetes Status']
    encoder = OneHotEncoder(sparse_output=False, drop='first') 
    encoded_cats = pd.DataFrame(
        encoder.fit_transform(data[categorical_cols]),
        columns=encoder.get_feature_names_out(categorical_cols)
    )

    # Drop original categorical columns and merge encoded ones
    data = pd.concat([data.drop(columns=categorical_cols), encoded_cats], axis=1)

    return data

In [305]:
demo_viome_train = preprocess_viome(demo_viome_train)
final_data = pd.merge(data_train, demo_viome_train, on=['Subject ID'])

final_data

,Subject ID,Day,Image Before Breakfast,Image Before Lunch,Breakfast Time,Lunch Time,CGM Data,Lunch Calories,Time Between Meals,Age,...,Viome_21,Viome_22,Viome_23,Viome_24,Viome_25,Viome_26,Race_Hispanic/Latino,Race_White,Diabetes Status_2.0,Diabetes Status_3.0
0,1,2,"[[[0.5490196078431373, 0.47843137254901963, 0....","[[[0.1607843137254902, 0.596078431372549, 0.78...",2021-09-19 08:41:00,2021-09-19 12:24:00,"[('2021-09-19 08:20:00', 98.26666666666667), (...",830,13380.0,27.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
1,1,3,"[[[0.2627450980392157, 0.22745098039215686, 0....","[[[0.1568627450980392, 0.23137254901960785, 0....",2021-09-20 09:50:00,2021-09-20 15:20:00,"[('2021-09-20 09:10:00', 97.18333333333334), (...",435,19800.0,27.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
2,1,4,"[[[0.7803921568627451, 0.7647058823529411, 0.7...","[[[0.20784313725490197, 0.17254901960784313, 0...",2021-09-21 09:34:00,2021-09-21 13:09:00,"[('2021-09-21 09:20:00', 107.36666666666666), ...",555,12900.0,27.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
3,1,5,"[[[0.5843137254901961, 0.4745098039215686, 0.3...","[[[0.11764705882352941, 0.10980392156862745, 0...",2021-09-22 09:46:00,2021-09-22 13:50:00,"[('2021-09-22 09:25:00', 107.28333333333333), ...",355,14640.0,27.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
4,1,6,"[[[0.6862745098039216, 0.7215686274509804, 0.7...","[[[0.2901960784313726, 0.3333333333333333, 0.3...",2021-09-23 09:07:00,2021-09-23 13:17:00,"[('2021-09-23 08:55:00', 103.0), ('2021-09-23 ...",1180,15000.0,27.0,...,0.773843,-0.125457,-0.352396,-0.241578,-0.135894,-0.164389,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,7,6,"[[[0.26666666666666666, 0.13333333333333333, 0...","[[[0.35294117647058826, 0.30196078431372547, 0...",2021-12-18 08:52:00,2021-12-18 12:28:00,"[('2021-12-18 08:50:00', 101.36), ('2021-12-18...",1180,12960.0,66.0,...,-0.179446,0.263283,0.491576,0.502913,-0.141314,-0.414110,1.0,0.0,1.0,0.0
315,7,7,"[[[0.10196078431372549, 0.10196078431372549, 0...","[[[0.06666666666666667, 0.03529411764705882, 0...",2021-12-19 08:43:00,2021-12-19 13:13:00,"[('2021-12-19 08:40:00', 100.68), ('2021-12-19...",830,16200.0,66.0,...,-0.179446,0.263283,0.491576,0.502913,-0.141314,-0.414110,1.0,0.0,1.0,0.0
316,7,8,"[[[0.16862745098039217, 0.1450980392156863, 0....","[[[0.47843137254901963, 0.4235294117647059, 0....",2021-12-20 09:06:00,2021-12-20 12:46:00,"[('2021-12-20 09:00:00', 104.04), ('2021-12-20...",435,13200.0,66.0,...,-0.179446,0.263283,0.491576,0.502913,-0.141314,-0.414110,1.0,0.0,1.0,0.0
317,7,9,"[[[0.1607843137254902, 0.14901960784313725, 0....","[[[0.23137254901960785, 0.1803921568627451, 0....",2021-12-21 08:34:00,2021-12-21 12:38:00,"[('2021-12-21 08:25:00', 96.4), ('2021-12-21 0...",555,14640.0,66.0,...,-0.179446,0.263283,0.491576,0.502913,-0.141314,-0.414110,1.0,0.0,1.0,0.0


### Dimensionality Reduction - PCA

In [306]:
# Normalize all numerical data
numeric_cols = final_data.select_dtypes(include=[np.number]).columns.difference(['Subject ID', 'Day', 'Lunch Calories'])
scaler = StandardScaler()
final_data[numeric_cols] = scaler.fit_transform(final_data[numeric_cols])

viome_columns = final_data.filter(like='Viome_')

# Perform PCA
pca = PCA()
X_pca = pca.fit_transform(viome_columns)

# Select the number of components that explain at least 90% of the variance
n_components = sum(pca.explained_variance_ratio_.cumsum() <= 0.90)
print(f"Number of components to explain 90% variance: {n_components}")

# Apply PCA with the selected number of components
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(viome_columns)

# Create a DataFrame with reduced features, preserving the component names
columns = [f"PC{i+1}" for i in range(n_components)]
reduced_df = pd.DataFrame(X_reduced, columns=columns, index=viome_columns.index)

# Select non-numerical and excluded columns from the original data
excluded_cols = final_data.drop(columns=viome_columns)

# Combine reduced features with the excluded columns, keeping column names intact
final_data = pd.concat([excluded_cols, reduced_df], axis=1)

Number of components to explain 90% variance: 20


In [307]:
final_data

,Subject ID,Day,Image Before Breakfast,Image Before Lunch,Breakfast Time,Lunch Time,CGM Data,Lunch Calories,Time Between Meals,Age,...,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20
0,1,2,"[[[0.5490196078431373, 0.47843137254901963, 0....","[[[0.1607843137254902, 0.596078431372549, 0.78...",2021-09-19 08:41:00,2021-09-19 12:24:00,"[('2021-09-19 08:20:00', 98.26666666666667), (...",830,-0.713046,-2.133138,...,2.289984,1.781071,0.699088,0.265082,1.197377,0.281796,-1.025412,0.825407,-1.123331,-0.548988
1,1,3,"[[[0.2627450980392157, 0.22745098039215686, 0....","[[[0.1568627450980392, 0.23137254901960785, 0....",2021-09-20 09:50:00,2021-09-20 15:20:00,"[('2021-09-20 09:10:00', 97.18333333333334), (...",435,0.997265,-2.133138,...,2.289984,1.781071,0.699088,0.265082,1.197377,0.281796,-1.025412,0.825407,-1.123331,-0.548988
2,1,4,"[[[0.7803921568627451, 0.7647058823529411, 0.7...","[[[0.20784313725490197, 0.17254901960784313, 0...",2021-09-21 09:34:00,2021-09-21 13:09:00,"[('2021-09-21 09:20:00', 107.36666666666666), ...",555,-0.840920,-2.133138,...,2.289984,1.781071,0.699088,0.265082,1.197377,0.281796,-1.025412,0.825407,-1.123331,-0.548988
3,1,5,"[[[0.5843137254901961, 0.4745098039215686, 0.3...","[[[0.11764705882352941, 0.10980392156862745, 0...",2021-09-22 09:46:00,2021-09-22 13:50:00,"[('2021-09-22 09:25:00', 107.28333333333333), ...",355,-0.377378,-2.133138,...,2.289984,1.781071,0.699088,0.265082,1.197377,0.281796,-1.025412,0.825407,-1.123331,-0.548988
4,1,6,"[[[0.6862745098039216, 0.7215686274509804, 0.7...","[[[0.2901960784313726, 0.3333333333333333, 0.3...",2021-09-23 09:07:00,2021-09-23 13:17:00,"[('2021-09-23 08:55:00', 103.0), ('2021-09-23 ...",1180,-0.281472,-2.133138,...,2.289984,1.781071,0.699088,0.265082,1.197377,0.281796,-1.025412,0.825407,-1.123331,-0.548988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,7,6,"[[[0.26666666666666666, 0.13333333333333333, 0...","[[[0.35294117647058826, 0.30196078431372547, 0...",2021-12-18 08:52:00,2021-12-18 12:28:00,"[('2021-12-18 08:50:00', 101.36), ('2021-12-18...",1180,-0.824936,1.471639,...,-0.363013,-1.014927,1.061170,-1.682789,0.880162,-2.196728,1.591268,-1.636508,-1.860606,1.113644
315,7,7,"[[[0.10196078431372549, 0.10196078431372549, 0...","[[[0.06666666666666667, 0.03529411764705882, 0...",2021-12-19 08:43:00,2021-12-19 13:13:00,"[('2021-12-19 08:40:00', 100.68), ('2021-12-19...",830,0.038212,1.471639,...,-0.363013,-1.014927,1.061170,-1.682789,0.880162,-2.196728,1.591268,-1.636508,-1.860606,1.113644
316,7,8,"[[[0.16862745098039217, 0.1450980392156863, 0....","[[[0.47843137254901963, 0.4235294117647059, 0....",2021-12-20 09:06:00,2021-12-20 12:46:00,"[('2021-12-20 09:00:00', 104.04), ('2021-12-20...",435,-0.760999,1.471639,...,-0.363013,-1.014927,1.061170,-1.682789,0.880162,-2.196728,1.591268,-1.636508,-1.860606,1.113644
317,7,9,"[[[0.1607843137254902, 0.14901960784313725, 0....","[[[0.23137254901960785, 0.1803921568627451, 0....",2021-12-21 08:34:00,2021-12-21 12:38:00,"[('2021-12-21 08:25:00', 96.4), ('2021-12-21 0...",555,-0.377378,1.471639,...,-0.363013,-1.014927,1.061170,-1.682789,0.880162,-2.196728,1.591268,-1.636508,-1.860606,1.113644


# Data Preparation

Multimodal dataset created for training combining breakfast and lunch images, cgm data and demographic data.

In [308]:
class MultimodalDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
        # Convert images to tensors and permute dimensions to [batch, channels, height, width]
        self.breakfast_images = torch.tensor(
            np.stack(data['Image Before Breakfast'].values), 
            dtype=torch.float32
        ).permute(0, 3, 1, 2)  # Change from NHWC to NCHW format
        
        self.lunch_images = torch.tensor(
            np.stack(data['Image Before Lunch'].values), 
            dtype=torch.float32
        ).permute(0, 3, 1, 2)  # Change from NHWC to NCHW format
        
        # Process CGM data
        def process_cgm(x):
            try:
                if isinstance(x, str):
                    clean_str = x.replace('[', '').replace(']', '').replace('\n', '').replace(' ', '')
                    if clean_str:
                        values = [float(i) for i in clean_str.split(',') if i]
                        if len(values) > 288:
                            return values[:288]
                        elif len(values) < 288:
                            return values + [0.0] * (288 - len(values))
                        return values
                return [0.0] * 288
            except:
                return [0.0] * 288
        
        processed_cgm = []
        for x in data['CGM Data'].values:
            cgm_values = process_cgm(x)
            processed_cgm.append(cgm_values)
            
        self.cgm_data = torch.tensor(processed_cgm, dtype=torch.float32)
        
        # Extract viome and demographic features
        viome_cols = [col for col in data.columns if 'PC' in col]
        # demo_viome_features = data[viome_cols].values

        demo_cols = ['Age', 'Weight', 'Height', 'A1C', 'Baseline Fasting Glucose', 
                     'Insulin', 'Triglycerides', 'Cholesterol', 'HDL', 'Non-HDL', 
                     'LDL', 'VLDL', 'CHO/HDL Ratio', 'HOMA-IR', 'BMI']
        
        demo_viome_features = data[viome_cols + demo_cols].values
        self.demo_viome = torch.tensor(demo_viome_features, dtype=torch.float32)
        
        # Extract labels
        self.labels = torch.tensor(data['Lunch Calories'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return {
            'breakfast_img': self.breakfast_images[idx],
            'lunch_img': self.lunch_images[idx],
            'cgm': self.cgm_data[idx],
            'demo_viome': self.demo_viome[idx],
            'label': self.labels[idx]
        }

# Multimodal Model Implementation

2 layer CNN

In [309]:
class MultimodalNet(nn.Module):
    def __init__(self, cgm_dim, demo_viome_dim):
        super(MultimodalNet, self).__init__()
        
        # Image encoders (modified for correct dimensions)
        self.image_encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Input: [B, 3, 64, 64]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: [B, 32, 32, 32]
            nn.Conv2d(32, 64, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: [B, 64, 16, 16]
            nn.Flatten(),                                # Output: [B, 64 * 16 * 16]
            nn.Linear(64 * 16 * 16, 256),               # Output: [B, 256]
            nn.ReLU()
        )
        # CGM encoder
        self.cgm_encoder = nn.Sequential(
            nn.Linear(288, 128),  # Assuming CGM length is 288
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        
        # Demo-Viome encoder
        self.demo_viome_encoder = nn.Sequential(
            nn.Linear(demo_viome_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Joint embedding
        self.fusion = nn.Sequential(
            nn.Linear(256 * 2 + 64 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, breakfast_img, lunch_img, cgm, demo_viome):
        # Ensure correct dimensions for images
        if breakfast_img.dim() == 3:
            breakfast_img = breakfast_img.unsqueeze(0)
        if lunch_img.dim() == 3:
            lunch_img = lunch_img.unsqueeze(0)
            
        # Encode images
        breakfast_features = self.image_encoder(breakfast_img)
        lunch_features = self.image_encoder(lunch_img)
        
        # Encode CGM
        cgm_features = self.cgm_encoder(cgm)
        
        # Encode demo-viome
        demo_viome_features = self.demo_viome_encoder(demo_viome)
        
        # Concatenate all features
        combined = torch.cat([
            breakfast_features, 
            lunch_features, 
            cgm_features, 
            demo_viome_features
        ], dim=1)
        
        # Final prediction
        output = self.fusion(combined)
        return output.squeeze()

In [310]:
class RMSRELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super(RMSRELoss, self).__init__()
        self.eps = eps
        
    def forward(self, pred, target):
        relative_error = (pred - target) / (target + self.eps)
        rmsre = torch.sqrt(torch.mean(relative_error ** 2))
        return rmsre

# Model Training

In [311]:
print("Shape of breakfast images:", final_data['Image Before Breakfast'].iloc[0].shape)
print("Number of Viome columns:", len([col for col in final_data.columns if 'PC' in col]))

dataset = MultimodalDataset(final_data)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model
cgm_dim = dataset.cgm_data.shape[1]  # Should be 288
demo_viome_dim = dataset.demo_viome.shape[1] 

model = MultimodalNet(
    cgm_dim=dataset.cgm_data.shape[1],
    demo_viome_dim=dataset.demo_viome.shape[1]
)

# Initialize optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
criterion = RMSRELoss()

# Training loop
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    
    for batch in loader:
        optimizer.zero_grad()
        
        pred = model(
            batch['breakfast_img'],
            batch['lunch_img'],
            batch['cgm'],
            batch['demo_viome']
        )
        
        loss = criterion(pred, batch['label'])
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

Shape of breakfast images: (64, 64, 3)
Number of Viome columns: 20


In [312]:
num_epochs = 5
for epoch in range(num_epochs):
    loss = train_epoch(model, train_loader, optimizer, criterion)
    print(f'Epoch {epoch+1}, Loss: {loss:.4f}')

Epoch 1, Loss: 0.7781
Epoch 2, Loss: 0.5203
Epoch 3, Loss: 0.4447
Epoch 4, Loss: 0.4315
Epoch 5, Loss: 0.3929


In [ ]:
# Prepare test data
temp_test_data = pd.merge(image_test, cgm_test, on=['Subject ID', 'Day'])
processed_test_data = preprocess_img(temp_test_data)
# processed_test_data = preprocess_cgm(processed_test_data)
demo_viome_test = preprocess_viome(demo_viome_test)
print(demo_viome_test.shape)
test_data = pd.merge(processed_test_data, demo_viome_test, on=['Subject ID'])
print(test_data.shape)

# Normalize all numerical data
numeric_cols = test_data.select_dtypes(include=[np.number]).columns.difference(['Subject ID', 'Day', 'Lunch Calories'])
test_data[numeric_cols] = scaler.fit_transform(test_data[numeric_cols])

viome_columns = test_data.filter(like='Viome_')

# Perform PCA
X_test_pca = pca.fit_transform(viome_columns)

# Apply PCA with the selected number of components
X_test_reduced = pca.fit_transform(viome_columns)

# Create a DataFrame with reduced features, preserving the component names
columns = [f"PC{i+1}" for i in range(n_components)]
reduced_df = pd.DataFrame(X_test_reduced, columns=columns, index=viome_columns.index)

# Select non-numerical and excluded columns from the original data
excluded_cols = test_data.drop(columns=viome_columns)

# Combine reduced features with the excluded columns, keeping column names intact
final_test_data = pd.concat([excluded_cols, reduced_df], axis=1)

print(final_test_data.shape)

(9, 48)
(73, 55)
(73, 48)


In [314]:
# Test Dataset Class
class MultimodalTestDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.breakfast_images = torch.tensor(
            np.stack(data['Image Before Breakfast'].values), dtype=torch.float32
        ).permute(0, 3, 1, 2)
        self.lunch_images = torch.tensor(
            np.stack(data['Image Before Lunch'].values), dtype=torch.float32
        ).permute(0, 3, 1, 2)

        viome_cols = [col for col in data.columns if 'PC' in col]
        # demo_viome_features = data[viome_cols].values
        # self.demo_viome = torch.tensor(demo_viome_features, dtype=torch.float32)

        demo_cols = ['Age', 'Weight', 'Height', 'A1C', 'Baseline Fasting Glucose', 
                     'Insulin', 'Triglycerides', 'Cholesterol', 'HDL', 'Non-HDL', 
                     'LDL', 'VLDL', 'CHO/HDL Ratio', 'HOMA-IR', 'BMI']
        demo_viome_features = data[viome_cols + demo_cols].values
        self.demo_viome = torch.tensor(demo_viome_features, dtype=torch.float32)
            
        def process_cgm(x):
            try:
                if isinstance(x, str):
                    clean_str = x.replace('[', '').replace(']', '').replace('\n', '').replace(' ', '')
                    if clean_str:
                        values = [float(i) for i in clean_str.split(',') if i]
                        if len(values) > 288:
                            return values[:288]
                        elif len(values) < 288:
                            return values + [0.0] * (288 - len(values))
                        return values
                return [0.0] * 288
            except:
                return [0.0] * 288
        
        processed_cgm = []
        for x in data['CGM Data'].values:
            cgm_values = process_cgm(x)
            processed_cgm.append(cgm_values)
            
        self.cgm_data = torch.tensor(processed_cgm, dtype=torch.float32)
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'breakfast_img': self.breakfast_images[idx],
            'lunch_img': self.lunch_images[idx],
            'cgm': self.cgm_data[idx],
            'demo_viome': self.demo_viome[idx]
        }

# Example usage for test data
test_dataset = MultimodalTestDataset(final_test_data)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Iterating through the test DataLoader
for batch in test_dataloader:
    print(batch['breakfast_img'].shape)
    print(batch['lunch_img'].shape)
    print(batch['demo_viome'].shape)
    print
    break  # Just show one batch for brevity

torch.Size([4, 3, 64, 64])
torch.Size([4, 3, 64, 64])
torch.Size([4, 35])


# Result Analysis

In [315]:
def evaluate_model(model, data_loader, criterion):
    """
    Evaluates the model on a given dataset.
    
    Args:
        model (nn.Module): The trained multimodal model.
        data_loader (DataLoader): DataLoader containing the test/validation dataset.
        criterion (nn.Module): Loss function used for evaluation.
        
    Returns:
        float: The average loss over the dataset.
        list: Predicted values for the dataset.
        list: Ground truth values for the dataset.
    """
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():  # Disable gradient computation for evaluation
        for batch in data_loader:
            # Extract input features and labels from the batch
            breakfast_img = batch['breakfast_img']
            lunch_img = batch['lunch_img']
            cgm = batch['cgm']
            demo_viome = batch['demo_viome']
            
            # Forward pass through the model
            outputs = model(breakfast_img, lunch_img, cgm, demo_viome)
            print(outputs)
            # Compute the loss
            # loss = criterion(outputs, labels)
            # total_loss += loss.item()
            if outputs.dim() == 0:  # Scalar tensor
                predictions.append(outputs.item())  # Convert scalar to Python float and append
            elif outputs.dim() == 1:  # 1D tensor
                predictions.extend(outputs.cpu().numpy().tolist()) 
            # Store predictions and true labels for later analysis
            # predictions.extend(outputs.cpu().numpy())
            # true_labels.extend(labels.cpu().numpy())
    
    # Calculate the average loss
    # average_loss = total_loss / len(data_loader)
    return predictions

In [316]:
# Example: Evaluate the model on a validation/test dataset
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # Use your test dataset here

# Evaluate the model
test_predictions = evaluate_model(model, test_dataloader, criterion)

# # Print results
# print(f"Test Loss: {test_loss:.4f}")

# # Optionally, analyze predictions and ground truth
# for i, (pred, true) in enumerate(zip(test_predictions[:5], test_labels[:5])):
#     print(f"Sample {i + 1}: Predicted: {pred:.2f}, True: {true:.2f}")

tensor([396.0352, 415.5573, 390.4001, 420.3568])
tensor([465.3853, 423.6745, 433.0529, 466.0335])
tensor([448.0571, 448.5552, 469.2532, 433.5075])
tensor([437.9947, 487.5036, 480.1547, 493.2020])
tensor([427.0498, 418.2768, 433.6376, 392.4466])
tensor([458.8183, 462.5072, 455.4987, 367.7613])
tensor([441.2866, 469.9388, 419.6432, 344.7279])
tensor([455.2320, 467.8134, 426.7909, 410.0947])
tensor([390.3049, 459.5814, 444.5727, 424.5822])
tensor([468.4174, 390.5848, 441.3591, 444.7759])
tensor([471.7954, 443.4714, 460.2391, 440.9697])
tensor([442.9554, 393.0736, 403.9447, 454.3777])
tensor([460.7761, 454.0807, 426.7883, 429.5280])
tensor([450.6420, 451.0998, 461.6155, 444.7664])
tensor([464.3485, 432.3849, 444.5110, 459.5536])
tensor([462.2749, 451.2357, 436.2307, 461.3674])
tensor([472.6994, 485.8143, 454.7756, 433.3916])
tensor([458.2046, 445.4177, 456.8134, 388.7779])
tensor(397.1254)


In [317]:
df = pd.DataFrame({
    'row_id': range(len(test_predictions)),
    'label': test_predictions
})

# Save the DataFrame to a CSV file
output_file = "test_predictions.csv"
df.to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}")

Predictions saved to test_predictions.csv
